In [ ]:
import torch
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset,Subset, random_split
import torchvision
from torchvision import datasets, models
from torchvision import transforms as T
import torchvision.transforms.functional as F
import torch.nn as nn
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import matplotlib.pyplot as plt
from IPython.display import display
import lightning as L
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks import ModelCheckpoint
import torchmetrics, argparse

from torchvision.datasets import ImageFolder

from PIL import Image
import os

import multiprocessing
num_workers = multiprocessing.cpu_count()
print(num_workers)
import timm
import wandb

In [ ]:
wandb.login()

In [40]:
class CFG:
    ver = 1.6
    seed = 42
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    base_dir =  "/root/signate_tecno/"
    input_dir = base_dir + "input/train"
    test_dir = base_dir + "input/test"
    output_dir = base_dir + "output/"
    sub_dir  = base_dir + "submit/"
    log_dir = base_dir + "logs/"
    model_dir = base_dir + "model/"
    ckpt_dir = base_dir + "ckpt/"

    MODEL = "eva_giant"
    DATASET = "TECNO"


    learning_rate = 1e-3
    weight_decay = 1e-5
    optimizer = "AdamW"
    data_aug = "RandAug"

In [ ]:
from pprint import pprint
pretrained_models = timm.list_models('eva*',pretrained=True)
pprint(pretrained_models)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

## train用のデータセット作成

In [51]:
class ImageDataset(Dataset):
    def __init__(self,data_dir,transform=None,phase = "train"):
        super().__init__()
        self.data_dir = data_dir
        self.image_paths = []
        self.labels = []
        for label in os.listdir(data_dir):
            label_dir = os.path.join(data_dir, label)
            for image_name in os.listdir(label_dir):
                image_path = os.path.join(label_dir, image_name)
                self.image_paths.append(image_path)
                self.labels.append(1 if label == "hold" else 0)
        self.transform = T.Compose([
                                        T.Resize((336,336)),
                                        T.RandomHorizontalFlip(p=0.5),
                                        T.ToTensor(),
                                        T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        self.phase = phase
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, index):
        image = Image.open(self.image_paths[index])
        label = self.labels[index]
        if self.transform:
            image = self.transform(image)
        return image, label

In [52]:
class LitDataModule(L.LightningDataModule):
    def __init__(self,batch_size = 32, data_dir = "./input", ds = None, data_aug = 'RandAug'):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.ds = ds

    def setup(self,stage = None):
        if stage == "fit" or stage is None:
            num_data = len(self.ds)
            print(f'data size: {num_data}')
            val_data_size = int(0.2 * num_data)
            train_data_size = num_data - val_data_size
            self.train_dataset, self.val_dataset = torch.utils.data.random_split(
                self.ds, [train_data_size, val_data_size], generator=torch.Generator().manual_seed(42)
            )
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers = num_workers, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers = num_workers)

In [ ]:
ds = ImageDataset(data_dir = CFG.input_dir)
dm = LitDataModule(data_dir = CFG.input_dir,batch_size = 32,ds = ds)
dm.prepare_data()
dm.setup(stage = "fit")

In [54]:
class ViTNet(L.LightningModule):
    def __init__(self,learning_rate = 1e-3, weight_decay = 1e-5, optimizer_name = "SGD", data_aug = "RandAug"):
        super().__init__()
        self.model = timm.create_model("eva_giant_patch14_336.clip_ft_in1k" , pretrained = True, num_classes = 2)
        # すべての層を固定
        for param in self.model.parameters():
            param.requires_grad = False

        # 'norm', 'fc_norm', 'head_drop', 'head' のみトレーニング可能にする
        for param in self.model.norm.parameters():
            param.requires_grad = True
        for param in self.model.fc_norm.parameters():
            param.requires_grad = True
        for param in self.model.head_drop.parameters():
            param.requires_grad = True
        for param in self.model.head.parameters():
            param.requires_grad = True
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.optimizer_name = optimizer_name
        self.data_aug = data_aug
        self.save_hyperparameters()
        self.acc = torchmetrics.classification.Accuracy(task= 'binary')
        self.class_acc = torchmetrics.classification.Accuracy(task = 'binary')
        self.loss_fn = nn.CrossEntropyLoss()
        self.predictions = []
        self.training_step_loss = []




    def forward(self,x):
        out = self.model(x)
        return out

    def _eval(self,batch,phase, on_step , on_epoch):
        x,y = batch
        out = self(x)
        loss = self.loss_fn(out, y)
        preds = torch.argmax(out, dim=1)
        acc = self.acc(preds, y)
        self.log(f"{phase}_loss", loss)
        self.log(f"{phase}_acc", acc, on_step = on_step, on_epoch = on_epoch)
        if phase == "val":
            self.class_acc(preds,y)
            self.log('hp_metric', acc, on_step = False, on_epoch = True,prog_bar = True, logger = True)
        return loss

    def training_step ( self,batch, batch_idx):
        loss = self._eval(batch, "train", on_step = False, on_epoch = True)
        self.training_step_loss.append(loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self._eval(batch, "val", on_step = False, on_epoch = True)
        return loss

    def on_train_epoch_end(self) -> None:
        all_loss = torch.stack(self.training_step_loss)
        self.log("train_epoch_loss", all_loss.mean())


    def configure_optimizers(self):
        if self.optimizer_name == "SGD":
            optimizer = optim.SGD(self.parameters(), lr = self.learning_rate, weight_decay = self.weight_decay)
        elif self.optimizer_name == "Adam":
            optimizer = optim.Adam(self.parameters(), lr = self.learning_rate, weight_decay = self.weight_decay)
        elif self.optimizer_name == "AdamW":
            optimizer = optim.AdamW(self.parameters(), lr = self.learning_rate, weight_decay = self.weight_decay)

        return optimizer

net = ViTNet(learning_rate = CFG.learning_rate,
             weight_decay = CFG.weight_decay,
             optimizer_name = CFG.optimizer,
             data_aug = CFG.data_aug)


In [ ]:
net.model

In [55]:
model_checkpoint = ModelCheckpoint(
    monitor='val_loss',
    dirpath=CFG.ckpt_dir,
    filename=f'{CFG.DATASET}-{CFG.ver}-{CFG.MODEL}'+ '-{epoch:02d}-{val_loss:.2f}',
    save_top_k=3,
    mode='min',
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=3,
)
from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger(name = f'eva_giant-{CFG.ver}',save_dir=CFG.log_dir)

In [56]:
callbacks = [model_checkpoint, early_stopping]

In [ ]:
trainer = L.Trainer(default_root_dir=CFG.log_dir,
                    max_epochs=10, logger=wandb_logger,
                    callbacks=callbacks)

In [58]:
import torch
torch.cuda.empty_cache()


In [ ]:
trainer.fit(net,datamodule = dm)

## test用のデータセット作成

In [ ]:
import pandas as pd
sample_submit = pd.read_csv(CFG.sub_dir + "sample_submit.csv", header=None)

In [ ]:
class TestDatset(Dataset):
    def __init__(self,df,data_dir,transform=None):
        super().__init__()
        self.df = df
        self.image_paths = df[0]
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        image_path = CFG.test_dir +"/"+self.image_paths[index]
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
class TestDataModule(L.LightningDataModule):
    def __init__(self,df,batch_size = 32, data_dir = "./input", ds = None):
        super().__init__()
        self.df = df
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.test_transform = T.Compose([
                                        T.Resize((224,224)),
                                        T.ToTensor(),
                                        T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    def setup(self,stage = None):
        if stage == "test" or stage is None:
            self.test_dataset = TestDatset(self.df,self.data_dir,self.test_transform)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers = num_workers)

In [ ]:
test_dm = TestDataModule(df = sample_submit,data_dir = CFG.test_dir)
test_dm.prepare_data()
test_dm.setup(stage = "test")

In [ ]:
test_dm.test_dataloader()

## checkpointからモデルをロードして推論

In [ ]:
checkpoint_path = "/content/drive/MyDrive/SIGNATE_TECNO/ckpt/TECNO-vit-epoch=03-val_loss=0.60.ckpt"
net = ViTNet.load_from_checkpoint(checkpoint_path)
net.eval()
net.freeze()


predict_list, targets_list = [], []

for process, images in enumerate(test_dm.test_dataloader()):
    images = images.to(CFG.device)
    with torch.no_grad():
        outputs = net(images)
        predicts = outputs.softmax(dim = 1)
    predicts = predicts.cpu().detach().numpy()
    predict_list.append(predicts)

predict_list = np.concatenate(predict_list,axis = 0)

predict_list = predict_list[:,1]

In [ ]:
file_count = len(os.listdir(CFG.test_dir))

print(f"Number of files in {CFG.test_dir}: {file_count}")

In [ ]:
sample_submit[1] = predict_list

In [ ]:
# prompt: predict_listを元に0,1に2値変換してください

predicted_labels = (predict_list > 0.5).astype(int)



In [ ]:
sample_submit[1] = predicted_labels

In [ ]:
sample_submit.head()

In [ ]:
sample_submit.to_csv(f"{CFG.sub_dir}/{CFG.ver}-{CFG.MODEL}-{CFG.seed}.csv", index=False, header=None)